<a href="https://colab.research.google.com/github/himalayahall/DATA608/blob/main/Story%201/Story1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

Password:

In [ ]:
!jupyter nbconvert --to pdf Story1.ipynb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
funding_ds = pd.read_csv("https://raw.githubusercontent.com/himalayahall/DATA608/main/Story%201/IIJA%20FUNDING%20AS%20OF%20MARCH%202023.csv")
funding_ds.head(n=10)

In [ ]:
funding_ds.dtypes

In [ ]:
funding_ds.rename(columns={"State, Teritory or Tribal Nation": "STATE", "Total (Billions)": "FUNDING"}, inplace=True)
funding_ds = funding_ds.astype({'STATE' : 'string'})
funding_ds.dtypes

In [ ]:
funding_ds = funding_ds[funding_ds['STATE'] != 'AMERICAN SAMOA']
funding_ds = funding_ds.reset_index(drop=True)
funding_ds.head(n=10)

In [ ]:
election2020_ds = pd.read_csv("https://raw.githubusercontent.com/himalayahall/DATA608/main/Story%201/2020_Presidential_Elections.csv", skip_blank_lines=True)
election2020_ds.head(n = 5)

In [ ]:
election2020_ds = election2020_ds.drop([0])
election2020_ds.head(n=10)

In [ ]:
election2020_ds.tail(n=10)

In [ ]:
election2020_ds = election2020_ds.drop([i for i in range(51, 56)])
election2020_ds = election2020_ds.reset_index(drop=True)
election2020_ds.tail(n=10)

In [ ]:
election2020_ds['BIDEN'] = election2020_ds['BIDEN'].str.replace('%', '', regex=False)
election2020_ds['TRUMP'] = election2020_ds['TRUMP'].str.replace('%', '', regex=False)
election2020_ds.head(n=10)

In [ ]:
election2020_ds['STATE'] = election2020_ds['STATE'].str.upper()
election2020_ds['STATE'] = election2020_ds['STATE'].str.replace('D.C.', 'DISTRICT OF COLUMBIA', regex=False)
election2020_ds.head(n=10)


In [ ]:
election2020_ds = election2020_ds.astype({'STATE': 'string', 'BIDEN': 'float', 'TRUMP' : 'float'})
election2020_ds.dtypes

In [ ]:
election2020_ds['BIDEN_WON'] = election2020_ds.apply(lambda row: 'YES' if row.BIDEN > row.TRUMP else 'NO', axis=1)
election2020_ds = election2020_ds.astype({'BIDEN_WON' : 'string'})

election2020_ds['COLOR'] = election2020_ds.apply(lambda row: 'blue' if row.BIDEN_WON == 'YES' else 'red', axis=1)
election2020_ds = election2020_ds.astype({'COLOR' : 'string'})
election2020_ds.head(10)

In [ ]:
population_ds = pd.read_csv("https://raw.githubusercontent.com/himalayahall/DATA608/main/Story%201/NST_EST2022_POP.csv", skip_blank_lines=True)
population_ds.head(n=10)

In [ ]:
population_ds = population_ds.drop([i for i in range(0, 8)])
population_ds = population_ds.reset_index(drop=True)
population_ds.head(n=10)

In [ ]:
population_ds = population_ds.drop(columns = [col for col in population_ds.columns if col in ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']])
population_ds.columns = ["STATE", "POPULATION"]
population_ds.tail(n=10)

In [ ]:
population_ds = population_ds.drop([i for i in range(51, 58)])

population_ds['STATE'] = population_ds['STATE'].str.replace('.', '', regex=False)
population_ds['STATE'] = population_ds['STATE'].str.upper()
population_ds = population_ds.astype({'STATE' : 'string'})

population_ds['POPULATION'] = population_ds['POPULATION'].str.replace(',', '', regex=False)
population_ds = population_ds.astype({'POPULATION' : 'int'})

population_ds = population_ds.reset_index(drop=True)
population_ds.tail(n=10)

In [ ]:
population_ds.dtypes

In [ ]:
plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = '20'
plt.xticks(rotation = 'vertical')
plt.bar(population_ds['STATE'],
        population_ds['POPULATION'])

In [ ]:
plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = '20'
plt.xticks(rotation = 'vertical')
plt.bar(funding_ds['STATE'],
        funding_ds['FUNDING'])

In [ ]:
funding_ds.head()

In [ ]:
comp = pd.merge(funding_ds, population_ds, how="outer", on=["STATE"], copy=True)
comp = comp.dropna()
comp = comp.astype({'POPULATION' : 'int'})
comp.head()

In [ ]:
merged = pd.merge(comp, election2020_ds, how="outer", on=["STATE"], copy=True)
merged = merged.dropna()
merged = merged.astype({'POPULATION' : 'int'})
merged = merged.drop(columns=["BIDEN", "TRUMP"])
merged.dtypes

In [ ]:
merged['PER_CAPITA_FUNDING'] = merged.apply(lambda row: (row.FUNDING * 1000000000) / row.POPULATION, axis=1)
merged.head(n=10)

In [ ]:
dems = len(merged[merged['BIDEN_WON'] == 'YES'])
reps = len(merged) - dems

plt.title(f'Per capita funding: {dems} Democratic, {reps} Republican')
plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = '20'
plt.xticks(rotation = 'vertical')
plt.xlabel('State')
plt.ylabel('Per capita funding')
plt.bar(merged['STATE'],
        merged['PER_CAPITA_FUNDING'],
        color = merged['COLOR'])


In [ ]:
top_n = 30
n_largest = merged.nlargest(top_n, 'PER_CAPITA_FUNDING')
dems = len(n_largest[n_largest['BIDEN_WON'] == 'YES'])
reps = len(n_largest) - dems

plt.title(f'Top {top_n} per capita funding: {dems} Democratic, {reps} Republican')
plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = '20'
plt.xticks(rotation = 'vertical')
plt.xlabel('State')
plt.ylabel('Per capita Funding')

plt.bar(n_largest['STATE'],
        n_largest['PER_CAPITA_FUNDING'],
        color = n_largest['COLOR'],
        label=['Republican' if s == 'NO' else 'Democratic' for s in n_largest['BIDEN_WON']])
